In [2]:
"""
You take the boat and find the gardener right where you were told he would be:
 managing a giant "garden" that looks more to you like a farm.

"A water source? Island Island is the water source!"
 You point out that Snow Island isn't receiving any water.

"Oh, we had to stop the water because we ran out of sand to filter it with!
 Can't make snow with dirty water. Don't worry, I'm sure we'll get more sand soon;
   we only turned off the water a few days... weeks... oh no." His face sinks into a look of horrified realization.

"I've been so busy making sure everyone here has food that I completely forgot to check why we stopped getting more sand!
 There's a ferry leaving soon that is headed over in that direction - it's much faster than your boat. Could you please go check it out?"

You barely have time to agree to this request when he brings up another.
 "While you wait for the ferry, maybe you can help us with our food production problem.
   The latest Island Island Almanac just arrived and we're having trouble making sense of it."

The almanac (your puzzle input) lists all of the seeds that need to be planted.
It also lists what type of soil to use with each kind of seed, 
what type of fertilizer to use with each kind of soil, what type of water to use with each kind of fertilizer, and so on.
 Every type of seed, soil, fertilizer and so on is identified with a number, but numbers are reused by each category -
   that is, soil 123 and fertilizer 123 aren't necessarily related to each other.

For example:

{full_example_input}

The almanac starts by listing which seeds need to be planted: seeds 79, 14, 55, and 13.

The rest of the almanac contains a list of maps which describe how to convert numbers 
from a source category into numbers in a destination category. 
That is, the section that starts with seed-to-soil map: describes how to convert a seed number (the source)
 to a soil number (the destination). This lets the gardener and his team know which soil to use with which seeds,
   which water to use with which fertilizer, and so on.

Rather than list every source number and its corresponding destination number one by one,
 the maps describe entire ranges of numbers that can be converted. Each line within a map contains three numbers:
   the destination range start, the source range start, and the range length.

Consider again the example seed-to-soil map:

50 98 2
52 50 48
The first line has a destination range start of 50, a source range start of 98, and a range length of 2.
 This line means that the source range starts at 98 and contains two values: 98 and 99.
   The destination range is the same length, but it starts at 50, so its two values are 50 and 51. With this information,
     you know that seed number 98 corresponds to soil number 50 and that seed number 99 corresponds to soil number 51.

The second line means that the source range starts at 50 and contains 48 values: 50, 51, ..., 96, 97.
 This corresponds to a destination range starting at 52 and also containing 48 values: 52, 53, ..., 98, 99. So, seed number 53 corresponds to soil number 55.

Any source numbers that aren't mapped correspond to the same destination number. So, seed number 10 corresponds to soil number 10.

So, the entire list of seed numbers and their corresponding soil numbers looks like this:

{example_input}

With this map, you can look up the soil number required for each initial seed number:

Seed number 79 corresponds to soil number 81.
Seed number 14 corresponds to soil number 14.
Seed number 55 corresponds to soil number 57.
Seed number 13 corresponds to soil number 13.
The gardener and his team want to get started as soon as possible,
so they'd like to know the closest location that needs a seed. Using these maps,
find the lowest location number that corresponds to any of the initial seeds. To do this,
you'll need to convert each seed number through other categories until you can find its corresponding location number.
 In this example, the corresponding types are:

Seed 79, soil 81, fertilizer 81, water 81, light 74, temperature 78, humidity 78, location 82.
Seed 14, soil 14, fertilizer 53, water 49, light 42, temperature 42, humidity 43, location 43.
Seed 55, soil 57, fertilizer 57, water 53, light 46, temperature 82, humidity 82, location 86.
Seed 13, soil 13, fertilizer 52, water 41, light 34, temperature 34, humidity 35, location 35.
So, the lowest location number in this example is 35.

What is the lowest location number that corresponds to any of the initial seed numbers?
"""

full_example_input ="""
seeds: 79 14 55 13

seed-to-soil map:
50 98 2
52 50 48

soil-to-fertilizer map:
0 15 37
37 52 2
39 0 15

fertilizer-to-water map:
49 53 8
0 11 42
42 0 7
57 7 4

water-to-light map:
88 18 7
18 25 70

light-to-temperature map:
45 77 23
81 45 19
68 64 13

temperature-to-humidity map:
0 69 1
1 0 69

humidity-to-location map:
60 56 37
56 93 4
"""

seed_soil_output = """
seed  soil
0     0
1     1
...   ...
48    48
49    49
50    52
51    53
...   ...
96    98
97    99
98    50
99    51
"""


In [8]:
full_example_input.split("map")

['\nseeds: 79 14 55 13\n\nseed-to-soil ',
 ':\n50 98 2\n52 50 48\n\nsoil-to-fertilizer ',
 ':\n0 15 37\n37 52 2\n39 0 15\n\nfertilizer-to-water ',
 ':\n49 53 8\n0 11 42\n42 0 7\n57 7 4\n\nwater-to-light ',
 ':\n88 18 7\n18 25 70\n\nlight-to-temperature ',
 ':\n45 77 23\n81 45 19\n68 64 13\n\ntemperature-to-humidity ',
 ':\n0 69 1\n1 0 69\n\nhumidity-to-location ',
 ':\n60 56 37\n56 93 4\n']

In [10]:
import numpy as np
import logging
import time
logging.basicConfig(level=logging.DEBUG)


logger = logging.getLogger("MapProcessor")
# clear previous handlers
logger.handlers = []
formatter = logging.Formatter('[%(name)s][%(levelname)-5s] %(message)s')
handler = logging.StreamHandler()
handler.setLevel(logging.DEBUG)
handler.setFormatter(formatter)
logger.addHandler(handler)

logger.propagate = False

class MapProcessor:
    def __init__(self, map_input, set_log_level=logging.INFO):
        if set_log_level:
            logger.setLevel(set_log_level)
        self.map_input = map_input
        self.map_type_index = {"seed": 0, "soil": 1, "fertilizer": 2, "water": 3, "light": 4, "temperature": 5, "humidity": 6, "location": 7}
        self.max_value = self.find_max_value()
        self.initial_seeds = self.get_map_array("seed")

        # 7xmax_value np array for all layers, each layer range is 0-max_value
        self.max_value = 2**27
        total_size = len(self.map_type_index) * self.max_value
        self.all_layers = np.arange(0,total_size,dtype=np.uint32).reshape(8, self.max_value) % self.max_value
        time.sleep(5)
        return
        # map all layers
        for map_type in ["soil", "fertilizer", "water", "light", "temperature", "humidity", "location"]:
            self.map_layer(map_type)

    def get_map_array(self, map_type):
        """ 
        Returns a map of the map_type, uses a dictionary to map the type to the index
        """
        map = self.map_input.split("map")
        map_type_index = self.map_type_index[map_type]
        if map_type_index == 0:
            map = map[0].splitlines()[1].split(" ")[1:]
            map = np.array(map, dtype=np.int32)
            logger.info(f"Initial seeds: {map[:10]}")
        else:
            map = map[map_type_index].splitlines()
            map = [line.split(" ") for line in map if line != "" and line[0].isdigit()]
            map = np.array(map, dtype=np.int32)
        # logger.debug(f"Adding map for {map_type} to cache")
        return map

    def find_max_value(self):
        """ Returns the max value across all maps
        Takes second value + length of map
        """
        max_value = 0
        for map_type in ["soil", "fertilizer", "water", "light", "temperature", "humidity", "location"]:
            map = self.get_map_array(map_type)
            max_value = (map[:,1] + map[:,2]).max() if (map[:,1] + map[:,2]).max() > max_value else max_value
        logger.info("Max value is: %s", max_value)
        return max_value

    def map_layer(self, map_type):
        """ Takes a layer and maps it against it the previous layer.
        """
        if map_type == "seed":
            logger.info("Skipping seed layer")
            return
        map_type_index = self.map_type_index[map_type]
        map_data  = self.get_map_array(map_type)
        logger.info(f"Type '{map_type}' ranges: \n {map_data}")
        # Start with the previous layer's values as the default for the current layer
        current_layer = self.all_layers[map_type_index - 1].copy()

        # Iterate over each range in the map data
        for dest_start, src_start, length in map_data:
            # Map each value in the source range to the corresponding value in the destination range
            for i in range(length):
                source_value = src_start + i
                destination_value = dest_start + i

                # Update the current layer's values where the source value matches
                current_layer[self.all_layers[map_type_index - 1] == source_value] = destination_value

        # Update the all_layers array with the new values for the current layer
        self.all_layers[map_type_index] = current_layer

    def view_initial_seed(self):
        """ Returns the corresponding locations for the initial seeds
        """
        return self.all_layers[:,self.initial_seeds]
    
    def print_layers(self,index_low=0,index_high=-1):
        """ Prints all layers with the map_type
        """
        logger.info(f"Printing all layers for index range {index_low}:{index_high}")
        for map_type, map_type_index in self.map_type_index.items():
            logger.info(f"Map type: {map_type}")
            logger.info(self.all_layers[map_type_index][index_low:index_high])

real_input = True
# Read the rucksack contents from the file
if real_input:
    with open("adventofcode.com_2023_day_5_input.txt", "r") as infile:
        content_input = infile.read()
else:
    content_input = full_example_input
processor = MapProcessor(full_example_input)
logger.warning(f"Memory usage: {processor.all_layers.nbytes/ (1024**3)} Gigabytes")

# processor.print_layers(75,85)
# initial_seed_maps = processor.view_initial_seed()
# logger.info(f"initial seeds maps:\n{initial_seed_maps}")

# Expected example output:
# Seed 79, soil 81, fertilizer 81, water 81, light 74, temperature 78, humidity 78, location 82.
# Seed 14, soil 14, fertilizer 53, water 49, light 42, temperature 42, humidity 43, location 43.
# Seed 55, soil 57, fertilizer 57, water 53, light 46, temperature 82, humidity 82, location 86.
# Seed 13, soil 13, fertilizer 52, water 41, light 34, temperature 34, humidity 35, location 35.
# So, the lowest location number in this example is 35.

[MapProcessor][WARNING] No full map in cache, splitting input
[MapProcessor][INFO ] Max value is: 100
[MapProcessor][INFO ] Initial seeds: [79 14 55 13]
[MapProcessor][WARNING] Memory usage: 4.0 Gigabytes
